In [9]:
import torch
import gym
import torch.nn as nn

In [12]:
class DQN(nn.Module):
    def __init__(self, n_actions, in_shape, hidden_size):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_shape, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

        
    
    def forward(self, X):
        return self.net(X)

    



In [17]:
from copy import copy
from random import random
import pygame
# ### Action Space
# There are four discrete actions available: do nothing, fire left
# orientation engine, fire main engine, fire right orientation engine.
# ### Observation Space
# The state is an 8-dimensional vector: the coordinates of the lander in `x` & `y`, its linear
# velocities in `x` & `y`, its angle, its angular velocity, and two booleans
# that represent whether each leg is in contact with the ground or not.
# ### Rewards
# After every step a reward is granted. The total reward of an episode is the
# sum of the rewards for all the steps within that episode.
# For each step, the reward:
# - is increased/decreased the closer/further the lander is to the landing pad.
# - is increased/decreased the slower/faster the lander is moving.
# - is decreased the more the lander is tilted (angle not horizontal).
# - is increased by 10 points for each leg that is in contact with the ground.
# - is decreased by 0.03 points each frame a side engine is firing.
# - is decreased by 0.3 points each frame the main engine is firing.
# The episode receive an additional reward of -100 or +100 points for crashing or landing safely respectively.
# An episode is considered a solution if it scores at least 200 points.
# ### Starting State
# The lander starts at the top center of the viewport with a random initial
# force applied to its center of mass.
# ### Episode Termination
# The episode finishes if:
# 1) the lander crashes (the lander body gets in contact with the moon);
# 2) the lander gets outside of the viewport (`x` coordinate is greater than 1);
# 3) the lander is not awake. From the [Box2D docs](https://box2d.org/documentation/md__d_1__git_hub_box2d_docs_dynamics.html#autotoc_md61),
#     a body which is not awake is a body which doesn't move and doesn't
#     collide with any other body:
# > When Box2D determines that a body (or group of bodies) has come to rest,
# > the body enters a sleep state which has very little CPU overhead. If a
# > body is awake and collides with a sleeping body, then the sleeping body
# > wakes up. Bodies will also wake up if a joint or contact attached to
# > them is destroyed.
import gym

eps = 0.001
env = gym.make("LunarLander-v2", render_mode="human")
observation_t, info = env.reset(seed=42)

agent = DQN(4,8,128)
double_agent = DQN(4,8,128)
buffer = []
num_samples = 3

loss_fn = torch.nn.MSELoss()
optim = torch.optim.Adam(agent.parameters(), lr=0.003)
c = 20

for _ in range(1):
    action = torch.argmax(agent(torch.Tensor(observation_t)))
    observation_t1, reward, terminated, truncated, info = env.step(action.item())
    
    buffer.append((observation_t if not terminated else terminated, action.item(), reward, observation_t1))
    observation_t = observation_t1

    for each in random.sample(buffer, num_samples):
        y_j = each[1] if each[0] is True else each[1] + torch.max(each[3])
        target = each[2] + torch.max(double_agent(each[3]))
        optim.zero_grad()
        loss = loss_fn(torch.Tensor(each[2]), target)
        loss.backward()
        optim.step()
    
    if c == 0:
        double_agent = copy.deepcopy(agent)
        c = 0
    c-=1


    # print(torch.Tensor(observation).shape)
    if terminated or truncated:
        observation, info = env.reset()
        break
env.close()
pygame.quit()

TypeError: __init__() missing 1 required positional argument: 'params'